# MWS Secondary Target Selection

## MWS_SPECIAL_RRLYR_SV

Maintainers:
  - Sergey Koposov (selection/catalog)
  - Andrew Cooper (secondary target code)

In [1]:
ls ../raw

gaiarr_source_ids_pos_pm.fits


We include all the stars from Gaia that were classified as RR Lyrae with mean G magnitudes between 14 and 19. Specifically that includes

1. All the stars from Gaia vari_rrlyrae table
2. All the stars from vari_classifier_result with RR* class

The selection criteria are:

```
WITH x as (SELECT vari_classifier_result.source_id
        FROM gaia_dr2.vari_classifier_result
        WHERE vari_classifier_result.best_class_name::text ~~ 'RR%'::text
    UNION
    SELECT vari_rrlyrae.source_id
        FROM gaia_dr2.vari_rrlyrae)
SELECT g.* FROM gaia_dr2.gaia_source as g, x
       where g.source_id = x.source_id
       and phot_g_mean_mag between 14 and 19;
```

**TODO: modify this to the query including other columns**

In [2]:
pwd

'/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_RRLYR_SV/1/code'

In [3]:
import os
import numpy as np
from astropy.table import Table, Column
import astropy.io.fits as fits

In [4]:
target_class   = 'MWS_SPECIAL_RRLYR_SV'
input_filename = 'gaiarr_source_ids_pos_pm.fits'

REF_CAT   = 'G'
REF_EPOCH = '2015.5'

In [5]:
# Relative paths
input_path  = os.path.join('..','raw',input_filename)

# Separate bright and faint output
output_path = os.path.join('..','indata','{}.fits'.format(target_class))

output_dir  = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    print('Making output directory: {}'.format(os.path.abspath(output_dir)))
    os.makedirs(dir)

In [6]:
print('Reading input: {}'.format(os.path.abspath(input_path)))
d = Table.read(input_path)
print('Read {:d} rows'.format(len(d)))

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_RRLYR_SV/1/raw/gaiarr_source_ids_pos_pm.fits
Read 228853 rows


In [7]:
print('First 3 rows:')
print(d[0:3])
print(d.colnames)

First 3 rows:
     source_id            ra           dec       ... ref_epoch     parallax   
------------------- ------------- -------------- ... --------- ---------------
4040430358516960896  268.58932269 -35.8625204223 ...    2015.5 -0.265084572626
6026375677963210240 253.064077285 -32.9826173643 ...    2015.5  0.193817322466
5393238919477110528 164.295037692 -40.4640310543 ...    2015.5  0.132261285167
['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'ref_epoch', 'parallax']


In [8]:
# No magnitude selection
bright_time_targets = np.arange(0,len(d))

Adjust to secondary target data model:

In [16]:
output = d['ra','dec','pmra','pmdec','ref_epoch','source_id'][bright_time_targets]

for cn_old, cn_new in zip(['ra','dec','pmra','pmdec','source_id','ref_epoch'],['RA','DEC','pmra','pmdec','REF_ID','REF_EPOCH']):
    output.rename_column(cn_old,cn_new)
    
output.add_column(Column(np.repeat(False,len(output)),'OVERRIDE'))
#output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMRA'))
#output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMDEC'))
output.add_column(Column(np.repeat(REF_CAT,len(output)),'REF_CAT'))
# output.add_column(Column(np.repeat(REF_EPOCH,len(output)),'REF_EPOCH'))

Add header:

In [17]:
output.meta['VRAWDATA'] = int(os.path.abspath(input_path).split(os.path.sep)[-3])

In [18]:
output[0:3]

RA,DEC,pmra,pmdec,REF_EPOCH,REF_ID,OVERRIDE,REF_CAT
float64,float64,float64,float64,float64,int64,bool,str1
268.58932269,-35.8625204223,-8.71972620514,-5.68536296795,2015.5,4040430358516960896,False,G
253.064077285,-32.9826173643,-0.884462014821,-6.05536628242,2015.5,6026375677963210240,False,G
164.295037692,-40.4640310543,-1.12221447784,-4.76469928441,2015.5,5393238919477110528,False,G


Write output:

In [19]:
print('Writing {:d} rows of output to {}'.format(len(output),os.path.abspath(output_path)))
output.write(output_path,overwrite=True)

Writing 228853 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_RRLYR_SV/1/indata/MWS_SPECIAL_RRLYR_SV.fits
